In [39]:
import librosa
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import multiprocessing as mp
from tqdm.notebook import tqdm

In [40]:
df_meta = pd.read_csv("./birdclef-2024/train_metadata.csv")

In [41]:
df_meta

,primary_label,secondary_labels,type,latitude,longitude,scientific_name,common_name,author,license,rating,url,filename
0,asbfly,[],['call'],39.2297,118.1987,Muscicapa dauurica,Asian Brown Flycatcher,Matt Slaymaker,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://www.xeno-canto.org/134896,asbfly/XC134896.ogg
1,asbfly,[],['song'],51.4030,104.6401,Muscicapa dauurica,Asian Brown Flycatcher,Magnus Hellström,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/164848,asbfly/XC164848.ogg
2,asbfly,[],['song'],36.3319,127.3555,Muscicapa dauurica,Asian Brown Flycatcher,Stuart Fisher,Creative Commons Attribution-NonCommercial-Sha...,2.5,https://www.xeno-canto.org/175797,asbfly/XC175797.ogg
3,asbfly,[],['call'],21.1697,70.6005,Muscicapa dauurica,Asian Brown Flycatcher,vir joshi,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/207738,asbfly/XC207738.ogg
4,asbfly,[],['call'],15.5442,73.7733,Muscicapa dauurica,Asian Brown Flycatcher,Albert Lastukhin & Sergei Karpeev,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://www.xeno-canto.org/209218,asbfly/XC209218.ogg
...,...,...,...,...,...,...,...,...,...,...,...,...
24454,zitcis1,[],[''],43.5925,4.5434,Cisticola juncidis,Zitting Cisticola,Chèvremont Fabian,Creative Commons Attribution-NonCommercial-Sha...,5.0,https://xeno-canto.org/845747,zitcis1/XC845747.ogg
24455,zitcis1,[],[''],43.5925,4.5434,Cisticola juncidis,Zitting Cisticola,Chèvremont Fabian,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/845817,zitcis1/XC845817.ogg
24456,zitcis1,[],[''],51.1207,4.5607,Cisticola juncidis,Zitting Cisticola,Wim Jacobs,Creative Commons Attribution-NonCommercial-Sha...,4.0,https://xeno-canto.org/856176,zitcis1/XC856176.ogg
24457,zitcis1,[],[''],41.5607,-8.4236,Cisticola juncidis,Zitting Cisticola,Jorge Leitão,Creative Commons Attribution-NonCommercial-Sha...,4.5,https://xeno-canto.org/856723,zitcis1/XC856723.ogg


In [42]:
df_train = df_meta[["primary_label", "filename"]]

In [43]:
def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam',
                  loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    return model

    

In [44]:
def extract_audio_features_with_path(args):
    ogg_file_path, max_length = args
    return extract_audio_features(ogg_file_path, max_length)

def extract_audio_features(ogg_file_path, max_length=22050*5):
    y, sr = librosa.load(ogg_file_path, sr=None)
    
    # Ensure the audio is of fixed length
    if len(y) < max_length:
        y = np.pad(y, (0, max_length - len(y)), 'constant')
    else:
        y = y[:max_length]
    
    # Extract features
    features = {}

    # Mel spectrogram
    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
    S_dB = librosa.power_to_db(S, ref=np.max)
    features['mel_spectrogram'] = S_dB

    # MFCC
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    features['mfcc'] = mfcc

    # Chroma feature
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    features['chroma'] = chroma

    # Spectral contrast
    spectral_contrast = librosa.feature.spectral_contrast(y=y, sr=sr)
    features['spectral_contrast'] = spectral_contrast

    # Tonnetz
    tonnetz = librosa.feature.tonnetz(y=librosa.effects.harmonic(y), sr=sr)
    features['tonnetz'] = tonnetz

    # Spectral centroid
    spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
    features['spectral_centroid'] = spectral_centroid

    # Spectral bandwidth
    spectral_bandwidth = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    features['spectral_bandwidth'] = spectral_bandwidth

    # Spectral rolloff
    spectral_rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
    features['spectral_rolloff'] = spectral_rolloff

    # Zero crossing rate
    zero_crossing_rate = librosa.feature.zero_crossing_rate(y)
    features['zero_crossing_rate'] = zero_crossing_rate

    # RMS
    rms = librosa.feature.rms(y=y)
    features['rms'] = rms

    return features


In [45]:

# Example usage
audio_features = extract_audio_features('./birdclef-2024/train_audio/asbfly/XC49755.ogg')

# Convert features to a dictionary of DataFrames for better visualization
features_df = {key: pd.DataFrame(value) for key, value in audio_features.items()}

# Display the extracted features
for feature_name, df in features_df.items():
    print(f"\nFeature: {feature_name}")
    display(df.head())  # Using display() from IPython.display for better visualization in Jupyter


Feature: mel_spectrogram


,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
0,-80.0,-80.0,-80.0,-80.0,-71.131088,-64.629059,-64.798882,-65.517334,-64.943108,-66.595901,...,-68.542099,-62.939053,-61.753540,-64.483032,-63.943794,-68.828384,-75.564575,-80.000000,-69.745300,-58.308228
1,-80.0,-80.0,-80.0,-80.0,-78.152901,-77.862320,-80.000000,-80.000000,-80.000000,-80.000000,...,-66.897995,-64.690155,-63.297081,-61.861908,-62.233330,-65.241440,-66.423111,-74.839699,-68.645683,-57.374542
2,-80.0,-80.0,-80.0,-80.0,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,...,-61.738266,-63.308044,-63.542503,-68.342384,-66.267365,-65.165871,-68.677277,-73.797104,-69.288116,-58.136024
3,-80.0,-80.0,-80.0,-80.0,-78.352684,-80.000000,-80.000000,-80.000000,-80.000000,-79.861069,...,-64.366867,-71.845642,-70.448776,-73.537903,-71.473465,-71.168381,-72.104736,-69.305649,-67.361259,-57.354889
4,-80.0,-80.0,-80.0,-80.0,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,-80.000000,...,-73.917900,-80.000000,-76.070633,-72.207756,-73.334656,-75.337105,-80.000000,-70.599350,-68.497894,-58.325111



Feature: mfcc


,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
0,-695.373413,-695.373413,-695.354370,-680.230347,-643.505615,-619.225769,-591.797607,-570.970276,-563.205383,-558.302429,...,-306.195435,-323.671509,-334.408813,-339.167542,-338.049377,-331.991302,-328.520233,-321.373627,-314.663361,-300.228394
1,0.000000,0.000000,0.000993,-1.203629,-5.267553,-6.745506,-6.516578,-5.824792,-6.088688,-6.661309,...,-41.372093,-18.021332,-13.957712,-14.097721,-14.342066,-13.824280,-19.953529,-27.281433,-24.620096,-22.640423
2,0.000000,0.000000,-0.026895,-17.487480,-48.594494,-65.481293,-84.124863,-95.908485,-98.228455,-101.852127,...,-174.743835,-175.672638,-179.184937,-179.671829,-181.474518,-182.583771,-186.912994,-188.352554,-176.718735,-133.417740
3,0.000000,0.000000,-0.002972,2.805940,12.572548,14.241904,11.571680,11.357303,12.281723,13.987467,...,65.480469,54.011528,45.415066,47.022861,52.940224,50.364700,52.326881,65.002045,67.655022,71.527222
4,0.000000,0.000000,0.026674,8.966082,11.528910,8.192678,3.498780,-2.270911,-9.443663,-8.712378,...,-91.412827,-79.573364,-68.684639,-62.239941,-48.788322,-53.536591,-59.001015,-60.851425,-54.280022,-25.470638



Feature: chroma


...

,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
0,13000.0,12156.25,10078.125,7750.0,6421.875,6343.75,6312.5,6078.125,6203.125,6015.625,...,6109.375,6093.75,5968.75,5796.875,5703.125,5515.625,5375.0,5375.0,5609.375,6031.25



Feature: zero_crossing_rate


,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
0,0.223633,0.297852,0.361328,0.295898,0.250488,0.234863,0.235352,0.245117,0.246582,0.242676,...,0.323242,0.3125,0.288086,0.258301,0.239258,0.218262,0.212891,0.219727,0.186035,0.133789



Feature: rms


,0,1,2,3,4,5,6,7,8,9,...,206,207,208,209,210,211,212,213,214,215
0,0.000013,0.000031,0.000073,0.000177,0.00027,0.000456,0.000616,0.000727,0.000869,0.000953,...,0.090907,0.066837,0.030106,0.029994,0.035177,0.04654,0.061427,0.067807,0.066481,0.057839


In [46]:
def aggregate_features(features):
    aggregated_features = {}
    for key, value in features.items():
        aggregated_features[key] = {
            'mean': np.mean(value, axis=1),
            'std': np.std(value, axis=1),
            'min': np.min(value, axis=1),
            'max': np.max(value, axis=1)
        }
    return aggregated_features

def format_features(aggregated_features):
    formatted_features = []
    for key in aggregated_features:
        for stat in aggregated_features[key]:
            formatted_features.extend(aggregated_features[key][stat])
    return np.array(formatted_features)

In [47]:
ogg_file_path = './birdclef-2024/train_audio/asbfly/XC49755.ogg'
max_length = 22050 * 5  # For example, 5 seconds at a sample rate of 22050 Hz

# Extract and aggregate features
features = extract_audio_features(ogg_file_path, max_length)
aggregated_features = aggregate_features(features)

# Format features for model input
formatted_features = format_features(aggregated_features)

print(formatted_features.shape)
print(formatted_features)


(684,)
[-6.78951569e+01 -7.15780640e+01 -7.08037949e+01 -7.23653183e+01
 -7.57353821e+01 -7.60923462e+01 -7.72794800e+01 -7.69549332e+01
 -7.69535065e+01 -7.48166962e+01 -7.59738770e+01 -7.50231094e+01
 -7.48816452e+01 -7.21270599e+01 -7.18847809e+01 -6.99784622e+01
 -6.92440033e+01 -6.64677048e+01 -6.55641098e+01 -6.29078407e+01
 -6.35348587e+01 -6.19041100e+01 -6.14968414e+01 -6.00429764e+01
 -5.98880310e+01 -5.87049522e+01 -5.84746971e+01 -5.72715187e+01
 -5.75240135e+01 -5.58471870e+01 -5.55498352e+01 -5.40977554e+01
 -5.38592224e+01 -5.26902885e+01 -5.27443390e+01 -5.14852333e+01
 -5.19623528e+01 -5.10422897e+01 -5.14217949e+01 -5.06474075e+01
 -5.04771423e+01 -4.96308174e+01 -5.02442932e+01 -4.85827980e+01
 -4.86090584e+01 -4.86892738e+01 -4.79537125e+01 -4.74637413e+01
 -4.74799500e+01 -4.70840187e+01 -4.66258774e+01 -4.66038132e+01
 -4.59247398e+01 -4.57369804e+01 -4.46376686e+01 -4.42834740e+01
 -4.41352196e+01 -4.43094559e+01 -4.40298653e+01 -4.37026901e+01
 -4.32755165e+01 -

In [48]:

def load_data(file_paths, labels, max_length=22050*5):
    X = []
    y = []

    for file_path, label in tqdm(zip(file_paths, labels), total=len(file_paths)):
        features = extract_audio_features(file_path, max_length)
        aggregated_features = aggregate_features(features)
        formatted_features = format_features(aggregated_features)
        X.append(formatted_features)
        y.append(label)
        print(len(X), len(X[-1]),y)
    return np.array(X), np.array(y)


In [32]:

# Assuming df_train is already loaded in your notebook
file_paths = df_train['filename'].apply(lambda x : "./birdclef-2024/train_audio/" + x ).tolist()
labels = df_train['primary_label'].astype('category').cat.codes.tolist()

# Load data
X, y = load_data(file_paths, labels)

  0%|          | 0/24459 [00:00<?, ?it/s]

1 684 [0]
2 684 [0, 0]
3 684 [0, 0, 0]
4 684 [0, 0, 0, 0]
5 684 [0, 0, 0, 0, 0]
6 684 [0, 0, 0, 0, 0, 0]
7 684 [0, 0, 0, 0, 0, 0, 0]
8 684 [0, 0, 0, 0, 0, 0, 0, 0]
9 684 [0, 0, 0, 0, 0, 0, 0, 0, 0]
10 684 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
11 684 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
12 684 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
13 684 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
14 684 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
15 684 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
16 684 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
17 684 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
18 684 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
19 684 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
20 684 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
21 684 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
22 684 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
23 684 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

KeyboardInterrupt: 

In [50]:
import numpy as np
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

def extract_and_format_features(file_path, max_length=22050*5):
    try:
        features = extract_audio_features(file_path, max_length)
        aggregated_features = aggregate_features(features)
        formatted_features = format_features(aggregated_features)
        return formatted_features
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

def process_batch(file_paths, labels, max_length):
    X_batch = []
    y_batch = []
    for file_path, label in zip(file_paths, labels):
        formatted_features = extract_and_format_features(file_path, max_length)
        if formatted_features is not None:
            X_batch.append(formatted_features)
            y_batch.append(label)
    return X_batch, y_batch

def load_data(file_paths, labels, max_length=22050*5, batch_size=1000):
    X = []
    y = []

    # Split the data into batches
    batches = [(file_paths[i:i + batch_size], labels[i:i + batch_size])
               for i in range(0, len(file_paths), batch_size)]

    # Process batches in parallel using ThreadPoolExecutor
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_batch, batch_files, batch_labels, max_length) 
                   for batch_files, batch_labels in batches]
        
        for future in tqdm(as_completed(futures), total=len(futures)):
            X_batch, y_batch = future.result()
            X.extend(X_batch)
            y.extend(y_batch)

    return np.array(X), np.array(y)

# Assuming df_train is already loaded in your notebook
file_paths = df_train['filename'].apply(lambda x: "./birdclef-2024/train_audio/" + x).tolist()
labels = df_train['primary_label'].astype('category').cat.codes.tolist()

# Load data
X, y = load_data(file_paths, labels)

print(f"Loaded {X.shape[0]} files.")
print(X.shape)
print(y.shape)


  0%|          | 0/25 [00:00<?, ?it/s]

C:\Users\Rami\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


Loaded 24459 files.
(24459, 684)
(24459,)


In [23]:
len(df_train)

24459

In [51]:
# save x and y as csv

np.savetxt('X.csv', X, delimiter=',')
np.savetxt('y.csv', y, delimiter=',')
print(X.shape)
print(y.shape)


(24459, 684)
(24459,)


In [22]:
print(X[1])

[-1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1.  1.  1.
  1.  1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1.  1. -1. -1. -1. -1. -1. -1.  1.  1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1.  1. -1. -1.  1. -1. -1.
 -1. -1. -1. -1. -1. -1.  1.  1.  1.  1.  1.  1.  1

In [52]:
# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert labels to categorical
y = to_categorical(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X.shape, y.shape)
# Define the neural network model
def create_model(input_shape):
    model = Sequential([
        Dense(256, input_shape=(input_shape,), activation='relu'),
        Dropout(0.5),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(64, activation='relu'),
        Dense(y_train.shape[1], activation='softmax')
    ])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create and train the model
model = create_model(X_train.shape[1])
# model.summary()

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

(24459, 684) (24459, 182)


C:\Users\Rami\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_8 (Dense)                 │ (None, 256)            │       175,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 182)            │        11,830 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 228,342 (891.96 KB)

 Trainable params: 228,342 (891.96 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
612/612 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.0416 - loss: 4.7455 - val_accuracy: 0.1502 - val_loss: 3.9708
Epoch 2/50
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1126 - loss: 3.9796 - val_accuracy: 0.1748 - val_loss: 3.6915
Epoch 3/50
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1412 - loss: 3.7625 - val_accuracy: 0.2018 - val_loss: 3.5608
Epoch 4/50
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1608 - loss: 3.6547 - val_accuracy: 0.2105 - val_loss: 3.4167
Epoch 5/50
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1783 - loss: 3.5467 - val_accuracy: 0.2269 - val_loss: 3.3921
Epoch 6/50
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1896 - loss: 3.4839 - val_accuracy: 0.2359 - val_loss: 3.3241
Epoch 7/50
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2017 - loss: 3.4291 - val_accuracy: 0.2357 - val_loss: 3.2860
Epoch 8/50
612/612 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.2099 - loss: 3.3823 - val_accuracy: 0.